In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

## Sea level daily data

In [2]:
df_sea = pd.read_csv('../input/charleston-sea-level-daily-dataset-1921-2014/daily Charleston 1921-2014.csv')
df_sea

,time,depth,latitude,longitude,sea_surface_height_above_reference_level,sensor_type_code
0,UTC,m,degrees_north,degrees_east,millimeters,NaN
1,1921-01-01T12:00:00Z,0.0,32.782,-79.925,NaN,NaN
2,1921-01-02T12:00:00Z,0.0,32.782,-79.925,NaN,NaN
3,1921-01-03T12:00:00Z,0.0,32.782,-79.925,NaN,NaN
4,1921-01-04T12:00:00Z,0.0,32.782,-79.925,NaN,NaN
...,...,...,...,...,...,...
34329,2014-12-27T12:00:00Z,0.0,32.782,-79.925,1813,1.0
34330,2014-12-28T12:00:00Z,0.0,32.782,-79.925,1752,1.0
34331,2014-12-29T12:00:00Z,0.0,32.782,-79.925,1692,1.0
34332,2014-12-30T12:00:00Z,0.0,32.782,-79.925,1789,1.0


In [3]:
df_sea.drop([0], inplace=True)
df_sea['date'] = pd.to_datetime(df_sea['time']).dt.to_period('D')
df_sea['height'] = pd.to_numeric(df_sea['sea_surface_height_above_reference_level'])
df_sea.drop(columns=['depth', 'latitude', 'longitude', 'sensor_type_code', 'sea_surface_height_above_reference_level', 'time'], inplace=True)

Временной ряд довольно длинный, я сократила его до последних десяти лет

In [4]:
df_sea = df_sea.iloc[len(df_sea)-365*10:]
df_sea

,date,height
30684,2005-01-03,1613.0
30685,2005-01-04,1598.0
30686,2005-01-05,1563.0
30687,2005-01-06,1547.0
30688,2005-01-07,1595.0
...,...,...
34329,2014-12-27,1813.0
34330,2014-12-28,1752.0
34331,2014-12-29,1692.0
34332,2014-12-30,1789.0


## Mean air temperature monthly data

In [5]:
df1 = pd.read_csv('../input/stockholm-mean-air-temperature-time-series/train_stockholm_monthly_mean_temperature.csv', sep=';')
df1.head(5)

,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,1980,-4.7,-6.2,-3.1,4.8,7.8,15.6,16.8,14.6,12.8,5.7,-0.0,-0.8
1,1981,-4.0,-2.7,-2.2,3.9,10.8,12.6,16.4,14.6,11.7,6.5,1.1,-5.7
2,1982,-7.2,-4.0,1.1,4.1,9.4,12.5,17.4,17.0,12.3,7.6,4.5,0.3
3,1983,0.7,-3.8,-0.1,4.0,10.2,14.0,18.0,17.8,12.4,7.5,1.2,-1.1
4,1984,-3.0,-1.7,-2.0,5.3,10.9,13.3,15.3,16.4,10.6,8.9,4.3,0.8


Данные переструктурированы в более удобный формат:

In [6]:
df1.columns = ['year', *list(np.arange(1, 13))]
new_df1 = dict()
new_df1['year'] = []
new_df1['month'] = list(np.arange(1, 13)) * len(df1)
new_df1['temp'] = []
for i in range(len(df1)):
    new_df1['year'] = new_df1['year'] + ([df1.iloc[i, 0]] * 12)
    new_df1['temp'] = new_df1['temp'] + list(df1.iloc[i, 1:])
new_df1 = pd.DataFrame.from_dict(new_df1)
new_df1

,year,month,temp
0,1980,1,-4.7
1,1980,2,-6.2
2,1980,3,-3.1
3,1980,4,4.8
4,1980,5,7.8
...,...,...,...
439,2016,8,16.1
440,2016,9,14.9
441,2016,10,6.9
442,2016,11,1.5


Оригинальные данные разделены на два датасета, train и test, я совместила их, чтобы разбить в своем соотношении

In [7]:
df2 = pd.read_csv('../input/stockholm-mean-air-temperature-time-series/test_stockholm_monthly_mean_temperature.csv', sep=';')
df2.columns = ['year', *list(np.arange(1, 13))]
new_df2 = dict()
new_df2['year'] = []
new_df2['month'] = list(np.arange(1, 13)) * len(df2)
new_df2['temp'] = []
for i in range(len(df2)):
    new_df2['year'] = new_df2['year'] + ([df2.iloc[i, 0]] * 12)
    new_df2['temp'] = new_df2['temp'] + list(df2.iloc[i, 1:])
new_df2 = pd.DataFrame.from_dict(new_df2)
df_air = new_df1.append(new_df2)
df_air['day'] = 1
df_air['date'] = pd.to_datetime(df_air[['day', 'month', 'year']])
df_air = df_air[['date', 'temp']]
df_air

,date,temp
0,1980-01-01,-4.7
1,1980-02-01,-6.2
2,1980-03-01,-3.1
3,1980-04-01,4.8
4,1980-05-01,7.8
...,...,...
43,2020-08-01,18.6
44,2020-09-01,13.8
45,2020-10-01,9.0
46,2020-11-01,6.4


In [8]:
df_sea.to_csv('sea_height.csv', index=False)
df_air.to_csv('temperature.csv', index=False)